In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
import numpy as np
import re
import json
import h2o

from datetime import datetime

from utils import get_data, Config, clean_url, make_clickable, add_columns

from tree import make_tree

In [4]:
# Load all data (except unknown/manual testing)
# Parsing + Basic 5x+ each, and reruns (such that all tests have 5+ results)
# Ubuntu 22.04, Android 11, MacOS 14.3.1, iPadOS 17.3.1
# All runs performed between 2024-03-04 and 2024-03-NN
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Browser".name != 'Unknown' and "Result".created_at > '2024-03-03';
"""
df = get_data(Config(), initial_data)
df = add_columns(df)

Connecting to the PostgreSQL database...
Connection successful


In [5]:
df["raw_header"] = df["raw_header"].apply(bytes).apply(lambda x: json.loads(x.decode("utf-8"))).astype(str)
def create_test_id(row):
    return f'{row["test_name"]}_{row["relation_info"]}_{row["org_scheme"]}_{row["org_host"]}_{row["resp_scheme"]}_{row["resp_host"]}_{row["response_id"]}_{row["resp_type"]}'

df["browser_id"] = df["browser_id"].astype("category")
# Takes a while (1000s+) (might be faster to already do it with postgres but not too important)
df["test_id"] = df.apply(create_test_id, axis=1)
df["test_id"] = df["test_id"].astype("category")

In [6]:
responses = """
SELECT * from "Response";
"""
responses = get_data(Config(), responses)

Connecting to the PostgreSQL database...
Connection successful


# Overview

In [7]:
# Remove other browsers/os
df = df.loc[~df["browser_id"].isin([30, 27, 26, 54, 28])]

In [9]:
# Ignore MacOS and iPad OS for now as they are not finished yet?!
df_m = df
df = df.loc[~df["browser_id"].isin([51, 60])]

In [10]:
# Both basic + parsing mode
# Each test should have results 5+ times (such that we can better reason about noise + majority voting makes sense)
df.groupby(["automation_mode", "browser_id", "name", "version", "os", "headless_mode"], observed=True)["id"].count().sort_values(ascending=False).to_frame().reset_index()

,automation_mode,browser_id,name,version,os,headless_mode,id
0,intent,5,opera,78.4.4143.75735,Android 11,real,1089526
1,intent,3,brave,1.60.116,Android 11,real,972918
2,intent,23,firefox_beta,122.0b1,Android 11,real,918988
3,selenium,47,chrome,122,Ubuntu 22.04,headless-new,914330
4,intent,12,chrome,120.0.6099.44,Android 11,real,906647
5,selenium,44,edge,121,Ubuntu 22.04,headless-new,897496
6,selenium,49,firefox,121,Ubuntu 22.04,headless,897384
7,selenium,48,chrome,120,Ubuntu 22.04,headless-new,897019
8,selenium,42,chrome,121,Ubuntu 22.04,headless-new,896516
9,selenium,45,brave,v1.62.156 (121.0.6167.139),Ubuntu 22.04,headless-new,896462


# Error + Timeout Analysis
- Iterative design:
  - We use higher timeouts for certain tests that timed out more often in debug runs
  - We use higher timeouts for Android
  - We use higher timeouts for the repeat runs
- Small number of random timeouts in both parsing/basic mode
- Very small number of not-run in Opera Android (HSTS)

In [11]:
# Timeout (2), Fail (1), Not-run (3)
filtered_df = df.groupby(["browser"])["test_status"].filter(lambda x: x.nunique() >= 2)
display(df[["browser", "test_status"]].loc[df.index.isin(filtered_df.index)].groupby(["browser"])["test_status"].value_counts().to_frame())

count
browser                                       test_status         
chrome Ubuntu 22.04 121 selenium headless-new 0             896513
                                              2                  3
chrome Ubuntu 22.04 122 selenium headless-new 0             914328
                                              2                  2
firefox Ubuntu 22.04 121 selenium headless    0             897065
                                              2                319
firefox Ubuntu 22.04 122 selenium headless    0             896295
                                              2                 17
firefox_beta Android 11 122.0b1 intent real   0             918846
                                              2                142
opera Android 11 78.4.4143.75735 intent real  0            1089423
                                              2                 79
                                              3                 24

In [12]:
# Teststatus == 2 (timeout)

# Rare timeouts in some browsers
display(df[df["test_status"] == 2]["browser"].value_counts().to_frame())

with pd.option_context("display.max_rows", 80):
    display(df.loc[(df["test_status"] == 2)].groupby(["resp_type", "status_code", "test_name"])["browser"].value_counts().to_frame().sort_values(by="count", ascending=False))

,count
browser,
firefox Ubuntu 22.04 121 selenium headless,319
firefox_beta Android 11 122.0b1 intent real,142
opera Android 11 78.4.4143.75735 intent real,79
firefox Ubuntu 22.04 122 selenium headless,17
chrome Ubuntu 22.04 121 selenium headless-new,3
chrome Ubuntu 22.04 122 selenium headless-new,2


count
resp_type status_code test_name                  browser                                             
parsing   200         fetch_GET                  firefox Ubuntu 22.04 121 selenium headless       153
basic     200         oac_iframe                 firefox_beta Android 11 122.0b1 intent real       73
                      referrer_iframe            firefox_beta Android 11 122.0b1 intent real       51
parsing   200         fetch_TEST                 firefox Ubuntu 22.04 121 selenium headless        51
                      subresourceloadingCOEP_img firefox Ubuntu 22.04 121 selenium headless        25
basic     200         fullscreen_iframe          opera Android 11 78.4.4143.75735 intent real      20
          302         framing_object             opera Android 11 78.4.4143.75735 intent real      18
                      framing_embed              opera Android 11 78.4.4143.75735 intent real      18
                      framing_iframe             opera Android 11 78.4.4143.75735 intent real      17
parsing   200         imgloading_iframe          firefox Ubuntu 22.04 121 selenium headless        15
basic     200         fetch_GET                  firefox Ubuntu 22.04 121 selenium headless        11
parsing   200         script_execution_iframe    firefox Ubuntu 22.04 121 selenium headless        10
                      fullscreen_iframe          firefox Ubuntu 22.04 121 selenium headless        10
basic     302         referrer_iframe            firefox_beta Android 11 122.0b1 intent real       10
parsing   200         framing_iframe             firefox Ubuntu 22.04 121 selenium headless         8
                      accesswindow_direct        firefox Ubuntu 22.04 121 selenium headless         6
basic     200         fetch_GET                  firefox Ubuntu 22.04 122 selenium headless         6
parsing   200         oac_window.open            firefox Ubuntu 22.04 121 selenium headless         5
                      script_execution_iframe    firefox Ubuntu 22.04 122 selenium headless         5
basic     200         referrer_iframe            firefox Ubuntu 22.04 121 selenium headless         5
                      fetch_TEST                 firefox Ubuntu 22.04 121 selenium headless         4
                      referrer_iframe            opera Android 11 78.4.4143.75735 intent real       4
                                                 firefox Ubuntu 22.04 122 selenium headless         4
parsing   200         accesswindow_direct        firefox_beta Android 11 122.0b1 intent real        4
          204         fetch_GET                  firefox Ubuntu 22.04 121 selenium headless         3
basic     200         fullscreen_iframe          chrome Ubuntu 22.04 121 selenium headless-new      3
parsing   300         fetch_GET                  firefox Ubuntu 22.04 121 selenium headless         3
          203         fetch_GET                  firefox Ubuntu 22.04 121 selenium headless         3
          200         fetch_GET                  firefox_beta Android 11 122.0b1 intent real        3
          201         fetch_GET                  firefox Ubuntu 22.04 121 selenium headless         3
basic     200         fetch_TEST                 firefox Ubuntu 22.04 122 selenium headless         2
                      oac_iframe                 opera Android 11 78.4.4143.75735 intent real       2
          302         script_execution_iframe    chrome Ubuntu 22.04 122 selenium headless-new      2
parsing   200         fetch_TEST                 firefox_beta Android 11 122.0b1 intent real        1
          203         fetch_TEST                 firefox Ubuntu 22.04 121 selenium headless         1
          201         fetch_TEST                 firefox Ubuntu 22.04 121 selenium headless         1
          204         fetch_TEST                 firefox Ubuntu 22.04 121 selenium headless         1
          300         fetch_TEST                 firefox Ubuntu 22.04 121 selenium headless         1

In [13]:
# Teststatus == 3 (not-run)

# Rare timeouts in all browsers; Safari some more than others
display(df[df["test_status"] == 3]["browser"].value_counts().to_frame())

# Mostly 302 redirects for basic (oac_iframe, oac_window.open), referrer_iframe 
# perfAPI_img/TAO (only Safari, fixed!)
# Others quite rare
display(df.loc[(df["test_status"] == 3)].groupby(["resp_type", "status_code", "test_name"])["browser"].value_counts().to_frame().sort_values(by="count", ascending=False))

,count
browser,
opera Android 11 78.4.4143.75735 intent real,24


count
resp_type status_code test_name             browser                                            
parsing   200         upgradeHSTS_subdomain opera Android 11 78.4.4143.75735 intent real     16
                      upgradeHSTS_direct    opera Android 11 78.4.4143.75735 intent real      8

# General Test Statistics

In [14]:
# Number of response_ids tested (should be 44415)
df.groupby("browser")["response_id"].nunique()

browser
brave Android 11 1.60.116 intent real                                  44415
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    44415
chrome Android 11 120.0.6099.44 intent real                            44415
chrome Ubuntu 22.04 120 selenium headless-new                          44415
chrome Ubuntu 22.04 121 selenium headless-new                          44415
chrome Ubuntu 22.04 122 selenium headless-new                          44415
edge Ubuntu 22.04 121 selenium headless-new                            44415
firefox Ubuntu 22.04 121 selenium headless                             44415
firefox Ubuntu 22.04 122 selenium headless                             44415
firefox_beta Android 11 122.0b1 intent real                            44415
opera Android 11 78.4.4143.75735 intent real                           44415
Name: response_id, dtype: int64

In [16]:
df.groupby("browser")["full_url"].nunique()

browser
brave Android 11 1.60.116 intent real                                  172485
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new     25109
chrome Android 11 120.0.6099.44 intent real                            128775
chrome Ubuntu 22.04 120 selenium headless-new                           25493
chrome Ubuntu 22.04 121 selenium headless-new                           25121
chrome Ubuntu 22.04 122 selenium headless-new                           28459
edge Ubuntu 22.04 121 selenium headless-new                             25618
firefox Ubuntu 22.04 121 selenium headless                              25513
firefox Ubuntu 22.04 122 selenium headless                              25109
firefox_beta Android 11 122.0b1 intent real                            151693
opera Android 11 78.4.4143.75735 intent real                           175334
Name: full_url, dtype: int64

In [17]:
# Number of test_ids
# Should be 10456 and 168774
df.loc[df["test_status"] == 0].groupby(["resp_type", "browser"])["test_id"].nunique()

resp_type  browser                                                            
basic      brave Android 11 1.60.116 intent real                                   10456
           brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new     10456
           chrome Android 11 120.0.6099.44 intent real                             10456
           chrome Ubuntu 22.04 120 selenium headless-new                           10456
           chrome Ubuntu 22.04 121 selenium headless-new                           10456
           chrome Ubuntu 22.04 122 selenium headless-new                           10456
           edge Ubuntu 22.04 121 selenium headless-new                             10456
           firefox Ubuntu 22.04 121 selenium headless                              10456
           firefox Ubuntu 22.04 122 selenium headless                              10456
           firefox_beta Android 11 122.0b1 intent real                             10456
           opera Android 11 78.

In [18]:
# Number of response_ids for each "group": resp_type, test_name, relation_info
df.groupby(["resp_type", "test_name", "relation_info"])["response_id"].nunique().to_frame()

response_id
resp_type test_name                     relation_info              
basic     accesswindow_direct           direct                   11
          fetch_GET                     credentials               6
                                        custom_headers            6
                                        simple                    6
          fetch_TEST                    custom_method             6
          framing_embed                 direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_iframe                direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_object                direct                   35
                                        nested                   35
                                        sandbox                  35
          fullscreen_iframe             child                    24
                                        child_allow              24
                                        child_sandbox            24
                                        direct                   24
          imgloading_iframe             direct                   17
          oac_iframe                    direct                   14
                                        sandbox                  14
          oac_window.open               window.open              14
          perfAPI_img                   direct                   13
          referrer_iframe               iframe                   16
                                        window.open              16
          script_execution_iframe       direct                   17
                                        sandbox                  17
          subresourceloadingCOEP_img    direct                   11
                                        sandbox                  11
          subresourceloadingCORP_img    direct                   11
                                        nested                   11
                                        sandbox                  11
          subresourceloadingCORP_object direct                   11
          upgradeHSTS_direct            direct                   14
          upgradeHSTS_subdomain         subdomain                14
parsing   accesswindow_direct           direct                 2656
          fetch_GET                     credentials            8059
                                        custom_headers         8059
                                        simple                 8059
          fetch_TEST                    custom_method          8059
          framing_iframe                direct                 8780
                                        nested                 8780
          fullscreen_iframe             child_allow            3880
                                        direct                 3880
          imgloading_iframe             direct                 2710
          oac_window.open               window.open            1699
          perfAPI_img                   direct                 1601
          referrer_iframe               iframe                 2797
          script_execution_iframe       direct                 2726
          subresourceloadingCOEP_img    direct                 2637
          subresourceloadingCORP_img    direct                 3357
          upgradeHSTS_direct            direct                 3324
          upgradeHSTS_subdomain         subdomain              3324

# Stability
- How many tests have more than one result for the 5+ repetitions in a browser?
- Basic:
    - total of 744/(10456*13) have different results
    - subresourceloadingCOEP_img is the worst offender (especially random behavior Firefox bug, but also in other browsers)
    - should be mostly timeout related and "fixed" by majority voting
- Parsing:
    - total of 17537/(168774*13) have different results
    - mostly brave Android: window.open did not work in 1+ repetition?
    - also a lot of Safari macOS: might be due to slow VPN? better to rerun in office?! @Tin
    - ...

In [19]:
# Remove all tests that timed out (test_status == 0), each test should have at least 5 results that did not time out in each browser
df_org = df
df = df.loc[df["test_status"] == 0]

In [20]:
details = False
for group_name, resp_type_group in df.groupby("resp_type"):
    print(group_name)
    stab = resp_type_group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])[["outcome_str"]].agg(["count", "nunique"])    # Tests with more than one outcome
    diff_outcomes = stab.loc[stab[("outcome_str", "nunique")] != 1]
    print(f"{len(diff_outcomes)} tests have different outcomes!")
    display(diff_outcomes.reset_index()["browser"].value_counts().to_frame())
    with pd.option_context("display.max_rows", 100):
        display(diff_outcomes.reset_index()[["test_name", "browser"]].value_counts().to_frame())

    diff_with_responses = diff_outcomes.reset_index().groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "browser"])["response_id"].agg(list).to_frame().reset_index()

    if not details:
        continue
    for id, (test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, browser, response_ids) in diff_with_responses.iterrows():
        print(test_name, response_ids)
        rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"].isin(response_ids)) & (df["browser"] == browser)]
        with pd.option_context("display.max_colwidth", 200):
            display(rows[["outcome_str", "browser"]].value_counts().to_frame())
            disp = rows.drop_duplicates(subset="outcome_str")[["browser", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
            disp = disp.style.format({'full_url': make_clickable})
            display(disp)

basic
743 tests have different outcomes!


,count
browser,
chrome Ubuntu 22.04 122 selenium headless-new,93
edge Ubuntu 22.04 121 selenium headless-new,89
firefox_beta Android 11 122.0b1 intent real,89
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,87
chrome Ubuntu 22.04 120 selenium headless-new,84
chrome Ubuntu 22.04 121 selenium headless-new,83
opera Android 11 78.4.4143.75735 intent real,79
firefox Ubuntu 22.04 121 selenium headless,73
firefox Ubuntu 22.04 122 selenium headless,60


count
(test_name, )              (browser, )                                              
subresourceloadingCOEP_img firefox_beta Android 11 122.0b1 intent real            89
                           firefox Ubuntu 22.04 121 selenium headless             69
                           firefox Ubuntu 22.04 122 selenium headless             52
                           brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     47
                           chrome Ubuntu 22.04 120 selenium headless-new          45
                           chrome Ubuntu 22.04 122 selenium headless-new          42
                           edge Ubuntu 22.04 121 selenium headless-new            39
                           chrome Ubuntu 22.04 121 selenium headless-new          37
accesswindow_direct        chrome Ubuntu 22.04 121 selenium headless-new          32
fullscreen_iframe          opera Android 11 78.4.4143.75735 intent real           30
accesswindow_direct        chrome Ubuntu 22.04 122 selenium headless-new          23
                           brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     21
                           chrome Ubuntu 22.04 120 selenium headless-new          20
fetch_GET                  edge Ubuntu 22.04 121 selenium headless-new            17
accesswindow_direct        edge Ubuntu 22.04 121 selenium headless-new            16
subresourceloadingCOEP_img opera Android 11 78.4.4143.75735 intent real           16
fetch_GET                  chrome Ubuntu 22.04 122 selenium headless-new          14
perfAPI_img                opera Android 11 78.4.4143.75735 intent real           12
fetch_GET                  chrome Ubuntu 22.04 121 selenium headless-new          10
                           chrome Ubuntu 22.04 120 selenium headless-new          10
                           brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      8
referrer_iframe            brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      6
imgloading_iframe          opera Android 11 78.4.4143.75735 intent real            6
fetch_GET                  opera Android 11 78.4.4143.75735 intent real            6
script_execution_iframe    chrome Ubuntu 22.04 120 selenium headless-new           5
                           edge Ubuntu 22.04 121 selenium headless-new             5
accesswindow_direct        chrome Android 11 120.0.6099.44 intent real             5
referrer_iframe            edge Ubuntu 22.04 121 selenium headless-new             5
script_execution_iframe    chrome Ubuntu 22.04 122 selenium headless-new           5
oac_window.open            brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      4
fetch_TEST                 edge Ubuntu 22.04 121 selenium headless-new             4
accesswindow_direct        firefox Ubuntu 22.04 122 selenium headless              4
referrer_iframe            chrome Ubuntu 22.04 122 selenium headless-new           4
upgradeHSTS_direct         opera Android 11 78.4.4143.75735 intent real            3
fetch_GET                  firefox Ubuntu 22.04 122 selenium headless              3
fetch_TEST                 opera Android 11 78.4.4143.75735 intent real            3
accesswindow_direct        opera Android 11 78.4.4143.75735 intent real            3
referrer_iframe            chrome Ubuntu 22.04 121 selenium headless-new           2
accesswindow_direct        firefox Ubuntu 22.04 121 selenium headless              2
fetch_TEST                 chrome Ubuntu 22.04 122 selenium headless-new           2
upgradeHSTS_subdomain      chrome Ubuntu 22.04 120 selenium headless-new           2
accesswindow_direct        brave Android 11 1.60.116 intent real                   1
fetch_GET                  firefox Ubuntu 22.04 121 selenium headless              1
script_execution_iframe    firefox Ubuntu 22.04 122 selenium headless              1
perfAPI_img                firefox Ubuntu 22.04 121 selenium headless              1
oac_window.open            edge Ubuntu 22.04 121 selenium headless-new             1

parsing
14415 tests have different outcomes!


,count
browser,
brave Android 11 1.60.116 intent real,8752
chrome Ubuntu 22.04 120 selenium headless-new,1830
firefox Ubuntu 22.04 121 selenium headless,1453
edge Ubuntu 22.04 121 selenium headless-new,889
chrome Ubuntu 22.04 121 selenium headless-new,411
firefox Ubuntu 22.04 122 selenium headless,329
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,252
chrome Ubuntu 22.04 122 selenium headless-new,215
opera Android 11 78.4.4143.75735 intent real,213


count
(test_name, )              (browser, )                                              
accesswindow_direct        brave Android 11 1.60.116 intent real                5312
oac_window.open            brave Android 11 1.60.116 intent real                3358
subresourceloadingCOEP_img chrome Ubuntu 22.04 120 selenium headless-new        1323
                           edge Ubuntu 22.04 121 selenium headless-new           677
fetch_GET                  firefox Ubuntu 22.04 121 selenium headless            545
fullscreen_iframe          firefox Ubuntu 22.04 121 selenium headless            544
imgloading_iframe          chrome Ubuntu 22.04 120 selenium headless-new         481
                           edge Ubuntu 22.04 121 selenium headless-new           177
fetch_TEST                 firefox Ubuntu 22.04 121 selenium headless            170
subresourceloadingCOEP_img chrome Ubuntu 22.04 122 selenium headless-new         165
upgradeHSTS_direct         firefox Ubuntu 22.04 122 selenium headless            163
                           chrome Ubuntu 22.04 121 selenium headless-new         139
upgradeHSTS_subdomain      chrome Ubuntu 22.04 121 selenium headless-new         136
                           brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...    132
                           firefox Ubuntu 22.04 122 selenium headless            125
upgradeHSTS_direct         brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...    117
subresourceloadingCOEP_img chrome Ubuntu 22.04 121 selenium headless-new         117
                           firefox Ubuntu 22.04 121 selenium headless            108
fetch_GET                  opera Android 11 78.4.4143.75735 intent real           85
upgradeHSTS_direct         brave Android 11 1.60.116 intent real                  62
subresourceloadingCOEP_img firefox Ubuntu 22.04 122 selenium headless             40
                           firefox_beta Android 11 122.0b1 intent real            36
upgradeHSTS_direct         opera Android 11 78.4.4143.75735 intent real           29
framing_iframe             firefox Ubuntu 22.04 121 selenium headless             28
fullscreen_iframe          opera Android 11 78.4.4143.75735 intent real           25
oac_window.open            firefox Ubuntu 22.04 121 selenium headless             24
framing_iframe             chrome Ubuntu 22.04 122 selenium headless-new          21
script_execution_iframe    firefox Ubuntu 22.04 121 selenium headless             21
referrer_iframe            opera Android 11 78.4.4143.75735 intent real           19
upgradeHSTS_direct         edge Ubuntu 22.04 121 selenium headless-new            19
                           chrome Android 11 120.0.6099.44 intent real            15
imgloading_iframe          chrome Ubuntu 22.04 121 selenium headless-new          15
upgradeHSTS_subdomain      brave Android 11 1.60.116 intent real                  15
fetch_TEST                 opera Android 11 78.4.4143.75735 intent real           14
script_execution_iframe    opera Android 11 78.4.4143.75735 intent real           13
upgradeHSTS_subdomain      opera Android 11 78.4.4143.75735 intent real           13
script_execution_iframe    chrome Ubuntu 22.04 120 selenium headless-new          11
fetch_GET                  chrome Ubuntu 22.04 120 selenium headless-new          10
fullscreen_iframe          chrome Ubuntu 22.04 122 selenium headless-new           9
imgloading_iframe          chrome Ubuntu 22.04 122 selenium headless-new           9
upgradeHSTS_subdomain      edge Ubuntu 22.04 121 selenium headless-new             9
upgradeHSTS_direct         chrome Ubuntu 22.04 122 selenium headless-new           9
subresourceloadingCOEP_img opera Android 11 78.4.4143.75735 intent real            8
framing_iframe             opera Android 11 78.4.4143.75735 intent real            7
upgradeHSTS_direct         firefox_beta Android 11 122.0b1 intent real             6
                           firefox Ubuntu 22.04 121 selenium headless              6

## More stability?!
- How many outcomes did we collect per test in the different browsers? Also shows tests with only one outcome
- How many of them are different?

In [21]:
# How many outcomes did we collect for each test?!
# Should be 5+ (less than 5 is concerning as we cannot do proper majority voting in such cases)

final_df = pd.DataFrame()

for group, d in df.groupby("browser"):
    # Calculate counts and unique counts
    aggs = ["count"] # ["count", "nunique"]
    # d = d.loc[d["org_scheme"] == "http"]
    res = d.groupby(["test_id"], observed=True)["outcome_str"].agg(aggs)
    # Get value counts for each combination of counts and unique counts
    counts = res.value_counts().to_frame()    
    counts = counts.rename(columns={"count": group})
    display(counts)
    final_df = pd.concat([final_df, counts], axis=1)

display(final_df)

,brave Android 11 1.60.116 intent real
count,
5,164138
11,11127
6,1409
9,723
7,714
8,688
10,402
12,29


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new
count,
5,178918
6,312


,chrome Android 11 120.0.6099.44 intent real
count,
5,175209
6,1528
7,758
10,621
8,553
9,500
17,22
18,12
16,11


,chrome Ubuntu 22.04 120 selenium headless-new
count,
5,178361
6,869


,chrome Ubuntu 22.04 121 selenium headless-new
count,
5,178896
6,305
7,29


...

,firefox Ubuntu 22.04 121 selenium headless
count,
5,178344
6,857
7,29


,firefox Ubuntu 22.04 122 selenium headless
count,
5,179121
6,73
7,36


,firefox_beta Android 11 122.0b1 intent real
count,
5,171947
9,3183
7,3078
8,602
6,270
11,89
10,45
66,13
65,3


,opera Android 11 78.4.4143.75735 intent real
count,
5,161148
14,2110
10,1956
17,1449
12,1313
13,1274
18,1236
15,1218
22,984


,brave Android 11 1.60.116 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 120.0.6099.44 intent real,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,firefox_beta Android 11 122.0b1 intent real,opera Android 11 78.4.4143.75735 intent real
count,,,,,,,,,,,
5,164138.0,178918.0,175209.0,178361.0,178896.0,167170.0,177884.0,178344.0,179121.0,171947.0,161148.0
11,11127.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,89.0,878.0
6,1409.0,312.0,1528.0,869.0,305.0,5942.0,1346.0,857.0,73.0,270.0,243.0
9,723.0,NaN,500.0,NaN,NaN,NaN,NaN,NaN,NaN,3183.0,668.0
7,714.0,NaN,758.0,NaN,29.0,6118.0,NaN,29.0,36.0,3078.0,72.0
8,688.0,NaN,553.0,NaN,NaN,NaN,NaN,NaN,NaN,602.0,575.0
10,402.0,NaN,621.0,NaN,NaN,NaN,NaN,NaN,NaN,45.0,1956.0
12,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1313.0
17,NaN,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1449.0


In [22]:
# Which tests have how many outcomes?

grouped = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type"])['outcome_str'].count().reset_index()

with pd.option_context("display.max_rows", 256):
    res = grouped.groupby(["browser", "resp_type", "test_name"])["outcome_str"].agg(["mean", "min", "max", "count", "sum"]).sort_values(by="mean", ascending=False)
    display(res.describe())
    display(res.loc[res["mean"] > 0].head(20))
    display(res.loc[res["mean"] > 0].tail(20))


,mean,min,max,count,sum
count,352.000000,352.000000,352.000000,352.000000,352.000000
mean,5.203962,5.042614,7.974432,5600.937500,28929.011364
std,0.552620,0.263446,7.552432,10295.584816,54243.302085
min,5.000000,5.000000,5.000000,28.000000,140.000000
25%,5.000000,5.000000,5.000000,284.000000,1440.000000
50%,5.000602,5.000000,6.000000,1680.000000,8400.000000
75%,5.069495,5.000000,7.000000,5487.500000,27972.500000
max,9.973214,7.000000,67.000000,48354.000000,370329.000000


mean  \
browser                                       resp_type test_name                                 
opera Android 11 78.4.4143.75735 intent real  basic     oac_window.open                9.973214   
                                              parsing   fetch_GET                      7.658705   
                                                        fetch_TEST                     7.658705   
                                              basic     fullscreen_iframe              7.299479   
firefox_beta Android 11 122.0b1 intent real   basic     referrer_iframe                7.218750   
chrome Ubuntu 22.04 122 selenium headless-new basic     upgradeHSTS_subdomain          7.000000   
                                                        subresourceloadingCORP_object  7.000000   
                                                        upgradeHSTS_direct             7.000000   
                                                        subresourceloadingCORP_img     7.000000   
                                                        subresourceloadingCOEP_img     7.000000   
                                                        script_execution_iframe        6.966912   
                                                        imgloading_iframe              6.941176   
                                                        accesswindow_direct            6.909091   
opera Android 11 78.4.4143.75735 intent real  basic     oac_iframe                     6.745536   
                                                        framing_iframe                 6.689881   
                                                        framing_object                 6.689286   
                                                        framing_embed                  6.689286   
brave Android 11 1.60.116 intent real         parsing   framing_iframe                 6.570843   
chrome Ubuntu 22.04 122 selenium headless-new basic     fetch_TEST                     6.500000   
                                                        fetch_GET                      6.500000   

                                                                                       min  \
browser                                       resp_type test_name                            
opera Android 11 78.4.4143.75735 intent real  basic     oac_window.open                  5   
                                              parsing   fetch_GET                        5   
                                                        fetch_TEST                       5   
                                              basic     fullscreen_iframe                5   
firefox_beta Android 11 122.0b1 intent real   basic     referrer_iframe                  5   
chrome Ubuntu 22.04 122 selenium headless-new basic     upgradeHSTS_subdomain            7   
                                                        subresourceloadingCORP_object    7   
                                                        upgradeHSTS_direct               7   
                                                        subresourceloadingCORP_img       7   
                                                        subresourceloadingCOEP_img       7   
                                                        script_execution_iframe          6   
                                                        imgloading_iframe                6   
                                                        accesswindow_direct              6   
opera Android 11 78.4.4143.75735 intent real  basic     oac_iframe                       5   
                                                        framing_iframe                   5   
                                                        framing_object                   5   
                                                        framing_embed                    5   
brave Android 11 1.60.116 intent real         parsing   framing_iframe                   5   
chrome Ubuntu 22.04 122 selenium

mean  \
browser                                       resp_type test_name                             
chrome Ubuntu 22.04 121 selenium headless-new parsing   accesswindow_direct             5.0   
                                                        fetch_GET                       5.0   
                                                        fetch_TEST                      5.0   
                                                        framing_iframe                  5.0   
                                                        fullscreen_iframe               5.0   
                                                        imgloading_iframe               5.0   
                                              basic     perfAPI_img                     5.0   
                                                        referrer_iframe                 5.0   
edge Ubuntu 22.04 121 selenium headless-new   parsing   upgradeHSTS_direct              5.0   
chrome Ubuntu 22.04 121 selenium headless-new basic     subresourceloadingCORP_img      5.0   
                                                        subresourceloadingCORP_object   5.0   
                                                        fetch_GET                       5.0   
                                                        fetch_TEST                      5.0   
edge Ubuntu 22.04 121 selenium headless-new   parsing   oac_window.open                 5.0   
                                                        perfAPI_img                     5.0   
                                                        referrer_iframe                 5.0   
                                                        script_execution_iframe         5.0   
                                                        subresourceloadingCOEP_img      5.0   
                                                        subresourceloadingCORP_img      5.0   
chrome Ubuntu 22.04 121 selenium headless-new basic     subresourceloadingCOEP_img      5.0   

                                                                                       min  \
browser                                       resp_type test_name                            
chrome Ubuntu 22.04 121 selenium headless-new parsing   accesswindow_direct              5   
                                                        fetch_GET                        5   
                                                        fetch_TEST                       5   
                                                        framing_iframe                   5   
                                                        fullscreen_iframe                5   
                                                        imgloading_iframe                5   
                                              basic     perfAPI_img                      5   
                                                        referrer_iframe                  5   
edge Ubuntu 22.04 121 selenium headless-new   parsing   upgradeHSTS_direct               5   
chrome Ubuntu 22.04 121 selenium headless-new basic     subresourceloadingCORP_img       5   
                                                        subresourceloadingCORP_object    5   
                                                        fetch_GET                        5   
                                                        fetch_TEST                       5   
edge Ubuntu 22.04 121 selenium headless-new   parsing   oac_window.open                  5   
                                                        perfAPI_img                      5   
                                                        referrer_iframe                  5   
                                                        script_execution_iframe          5   
                                                        subresourceloadingCOEP_img       5   
                                                        subresourceloadingCORP_img       5   
chrome Ubuntu 22.04 12

In [23]:
# Different outcomes (in percentage for a test group (test_name)
# (Does not take into account how often each test was executed)
grouped = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type"])['outcome_str'].nunique().reset_index()

grouped["diff_outcome"] = grouped["outcome_str"] > 1
with pd.option_context("display.max_rows", 100):
    res = grouped.groupby(["browser", "resp_type", "test_name"])["diff_outcome"].agg(["mean", "count", "sum"]).sort_values(by="mean", ascending=False)
    display(res.describe())
    display(res.loc[res["mean"] > 0].head(20))

,mean,count,sum
count,352.000000,352.000000,352.000000
mean,0.016310,5600.937500,43.062500
std,0.081058,10295.584816,346.523255
min,0.000000,28.000000,0.000000
25%,0.000000,284.000000,0.000000
50%,0.000000,1680.000000,0.000000
75%,0.001778,5487.500000,3.000000
max,1.000000,48354.000000,5312.000000


mean  \
browser                                            resp_type test_name                              
brave Android 11 1.60.116 intent real              parsing   accesswindow_direct         1.000000   
                                                             oac_window.open             0.988228   
firefox_beta Android 11 122.0b1 intent real        basic     subresourceloadingCOEP_img  0.252841   
chrome Ubuntu 22.04 120 selenium headless-new      parsing   subresourceloadingCOEP_img  0.250853   
firefox Ubuntu 22.04 121 selenium headless         basic     subresourceloadingCOEP_img  0.196023   
chrome Ubuntu 22.04 121 selenium headless-new      basic     accesswindow_direct         0.181818   
firefox Ubuntu 22.04 122 selenium headless         basic     subresourceloadingCOEP_img  0.147727   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... basic     subresourceloadingCOEP_img  0.133523   
chrome Ubuntu 22.04 122 selenium headless-new      basic     accesswindow_direct         0.130682   
edge Ubuntu 22.04 121 selenium headless-new        parsing   subresourceloadingCOEP_img  0.128366   
chrome Ubuntu 22.04 120 selenium headless-new      basic     subresourceloadingCOEP_img  0.127841   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... basic     accesswindow_direct         0.119318   
chrome Ubuntu 22.04 122 selenium headless-new      basic     subresourceloadingCOEP_img  0.119318   
chrome Ubuntu 22.04 120 selenium headless-new      basic     accesswindow_direct         0.113636   
edge Ubuntu 22.04 121 selenium headless-new        basic     subresourceloadingCOEP_img  0.110795   
opera Android 11 78.4.4143.75735 intent real       basic     upgradeHSTS_direct          0.107143   
chrome Ubuntu 22.04 121 selenium headless-new      basic     subresourceloadingCOEP_img  0.105114   
edge Ubuntu 22.04 121 selenium headless-new        basic     accesswindow_direct         0.090909   
chrome Ubuntu 22.04 120 selenium headless-new      parsing   imgloading_iframe           0.088745   
                                                   basic     upgradeHSTS_subdomain       0.071429   

                                                                                         count  \
browser                                            resp_type test_name                           
brave Android 11 1.60.116 intent real              parsing   accesswindow_direct          5312   
                                                             oac_window.open              3398   
firefox_beta Android 11 122.0b1 intent real        basic     subresourceloadingCOEP_img    352   
chrome Ubuntu 22.04 120 selenium headless-new      parsing   subresourceloadingCOEP_img   5274   
firefox Ubuntu 22.04 121 selenium headless         basic     subresourceloadingCOEP_img    352   
chrome Ubuntu 22.04 121 selenium headless-new      basic     accesswindow_direct           176   
firefox Ubuntu 22.04 122 selenium headless         basic     subresourceloadingCOEP_img    352   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... basic     subresourceloadingCOEP_img    352   
chrome Ubuntu 22.04 122 selenium headless-new      basic     accesswindow_direct           176   
edge Ubuntu 22.04 121 selenium headless-new        parsing   subresourceloadingCOEP_img   5274   
chrome Ubuntu 22.04 120 selenium headless-new      basic     subresourceloadingCOEP_img    352   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... basic     accesswindow_direct           176   
chrome Ubuntu 22.04 122 selenium headless-new      basic     subresourceloadingCOEP_img    352   
chrome Ubuntu 22.04 120 selenium headless-new      basic     accesswindow_direct           176   
edge Ubuntu 22.04 121 selenium headless-new        basic     subresourceloadingCOEP_img    352   
opera Android 11 78.4.4143.75735 intent real       basic     upgradeHSTS_direct             28   
chrome Ubuntu 22.04 121 selenium headless-new      basic     subresourceloadingCOEP_img    352   
edge Ubu

# Browser differences!
- First perform majority voting
- Then diff the browsers

In [24]:
df = df_org
df = df.loc[df["test_status"] == 0]

In [25]:
df_stab = df

# Only keep one row for each test
# If the test had non-deterministic results, use majority voting! (only works reliably if there a enough repetitions of the tests runs, >= 5)

print("Original data entries", len(df))

# Majority voting (pd.Series.mode returns the most frequest item) (quite slow)

df = df.groupby(["test_id", "browser"], observed=True)["outcome_str"].agg(pd.Series.mode).reset_index()
print("Only one row for each test (per browser; majority voting)", len(df))

Original data entries 10183012
Only one row for each test (per browser; majority voting) 1971530


In [26]:
# Merge back additional required properties!
# ["name", "test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "resp_type", "label", 'org_origin', 'resp_origin', 'status_code', 'raw_header']

df = df.merge(df_stab.drop_duplicates(subset=["test_id", "browser"]), on=["test_id", "browser"], how="left", suffixes=["", "_ignore"])


In [27]:
# If several values occur the same, the mode is an np.ndarray and not a string!
# If we have exactly 5 repetitions for each test, the issue cannot exist!
# However, our repeat system only enables us to have 5+ results for each test
print("Tests with the same mode (highest frequency outcome):")
display(df.loc[df['outcome_str'].apply(lambda x: isinstance(x, np.ndarray))].groupby(["browser", "resp_type", "test_name"])["outcome_str"].count())

# In the rare cases, where it actually matters, we can just take the first? (this might be noise in the generated trees and other summaries!)
# These are unstable tests that might indicate random behavior in the browser (e.g., firefox COEP_img)
df['outcome_str'] = df['outcome_str'].apply(lambda x: x[0] if isinstance(x, np.ndarray) else x)

Tests with the same mode (highest frequency outcome):


browser                                       resp_type  test_name                 
firefox_beta Android 11 122.0b1 intent real   basic      subresourceloadingCOEP_img    11
opera Android 11 78.4.4143.75735 intent real  parsing    upgradeHSTS_direct             1
Name: outcome_str, dtype: int64

In [28]:
# Most tests have exactly one outcome, quite some have two, couple have three!
res = df.groupby(["resp_type", "test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "test_id"], observed=True)
res = res["outcome_str"].agg("nunique")
display(res.value_counts().to_frame())

,count
outcome_str,
1,172157
2,3545
3,3528


In [29]:
# Basic:
# subresourceloadingCORP_object has different results for 100% of tests
# couple of others have 20%+ differences
# couple have 0%

# Parsing:
# oac_window.open has 99%
# perfAPI_img has 60% (might be an issue with the current macOS VPN based test run?)
# Other test groups have between 0.5 - 5% of tests with different outcomes
r = res.reset_index()
r["diff"] = r["outcome_str"] != 1
r.groupby(["resp_type", "test_name", "relation_info"])["diff"].agg(["count", "sum", "mean"]).sort_values("mean", ascending=False)

count   sum      mean
resp_type test_name                     relation_info                        
parsing   oac_window.open               window.open      3398  3370  0.991760
basic     oac_window.open               window.open       224   192  0.857143
          oac_iframe                    sandbox           224   144  0.642857
                                        direct            224   144  0.642857
          referrer_iframe               window.open       256   120  0.468750
          subresourceloadingCOEP_img    direct            176    77  0.437500
          fullscreen_iframe             direct            384   142  0.369792
                                        child_allow       384   113  0.294271
          perfAPI_img                   direct            208    52  0.250000
          subresourceloadingCORP_object direct            176    32  0.181818
          accesswindow_direct           direct            176    28  0.159091
          referrer_iframe               iframe            256    18  0.070312
          fullscreen_iframe             child             384    26  0.067708
          imgloading_iframe             direct            272    13  0.047794
parsing   fullscreen_iframe             child_allow      7760   355  0.045747
          imgloading_iframe             direct           5420   230  0.042435
basic     fetch_GET                     custom_headers     96     4  0.041667
          fetch_TEST                    custom_method      96     4  0.041667
          fetch_GET                     simple             96     4  0.041667
                                        credentials        96     4  0.041667
parsing   script_execution_iframe       direct           5452   216  0.039618
          upgradeHSTS_direct            direct           6648   250  0.037605
basic     upgradeHSTS_direct            direct             28     1  0.035714
parsing   fullscreen_iframe             direct           7760   272  0.035052
basic     subresourceloadingCOEP_img    sandbox           176     4  0.022727
parsing   framing_iframe                direct          17560   365  0.020786
          referrer_iframe               iframe           5594   102  0.018234
basic     framing_iframe                sandbox           560     8  0.014286
parsing   subresourceloadingCOEP_img    direct           5274    64  0.012135
          upgradeHSTS_subdomain         subdomain        6648    69  0.010379
          subresourceloadingCORP_img    direct           6714    56  0.008341
          framing_iframe                nested          17560   144  0.008200
basic     fullscreen_iframe             child_sandbox     384     3  0.007812
parsing   accesswindow_direct           direct           5312    40  0.007530
          fetch_GET                     simple          16118   104  0.006452
                                        custom_headers  16118   101  0.006266
          fetch_TEST                    custom_method   16118   101  0.006266
basic     subresourceloadingCORP_img    direct            176     1  0.005682
parsing   fetch_GET                     credentials     16118    86  0.005336
          perfAPI_img                   direct           3202    14  0.004372
basic     framing_embed                 nested            560     0  0.000000
          framing_iframe                direct            560     0  0.000000
                                        nested            560     0  0.000000
          framing_object                direct            560     0  0.000000
                                        nested            560     0  0.000000
                                        sandbox           560     0  0.000000
          framing_embed                 direct            560     0  0.000000
                                        sandbox           560     0  0.000000
          upgradeHSTS_subdomain         subdomain          28     0  0.000000
          script_execution_iframe       sandbox           272     0  0.000000


In [30]:
# Select only test rows with more than one outcome
tests_with_mult_outcomes = r.loc[r["diff"]].set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
df_mult = df.loc[df[["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]].apply(tuple, axis=1).isin(tests_with_mult_outcomes.index)]
tests_with_more_than_one_outcome = df_mult.drop_duplicates(subset=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])


all_tests_should_be =  df_org.loc[df_org["test_status"] == 0]["test_id"].nunique() * df_org["browser"].nunique() # (num parsing tests + num_basic_tests) * num_browsers (168774+10456)
print(f"All test rows: {len(df)}, should be: {all_tests_should_be}, Rows with more than one outcome: {len(df_mult)}, Tests with more than one outcome: {len(tests_with_more_than_one_outcome)}")

All test rows: 1971530, should be: 1971530, Rows with more than one outcome: 77803, Tests with more than one outcome: 7073


In [31]:
# Display difference groups
for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(grouping)
    df_list = []
    for _, one_test in group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]):
        browsers = one_test.groupby("outcome_str")["browser"].unique().apply(list).apply(sorted).to_frame()
        new = True
        for df_b in df_list:
            if df_b.equals(browsers):
                new = False
        if new:
            df_list.append(browsers)
    print(len(df_list))
    for df_b in df_list:
        with pd.option_context("display.max_colwidth", 1000):
            display(df_b)
    #input("Continue!")

('accesswindow_direct', 'direct')
5


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new]"
"{'window.open.opener': 'object ""[object Window]""'}","[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox_beta Android 11 122.0b1 intent real, opera Android 11 78.4.4143.75735 intent real]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, opera Android 11 78.4.4143.75735 intent real]"
"{'window.open.opener': 'object ""[object Window]""'}","[brave Android 11 1.60.116 intent real, chrome Android 11 120.0.6099.44 intent real, firefox_beta Android 11 122.0b1 intent real]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 78.4.4143.75735 intent real]"
"{'window.open.opener': 'object ""[object Window]""'}","[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox_beta Android 11 122.0b1 intent real]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox_beta Android 11 122.0b1 intent real]"
"{'window.open.opener': 'object ""[object Window]""'}","[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 78.4.4143.75735 intent real]"


...

,browser
outcome_str,
Fetch timed out,"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox_beta Android 11 122.0b1 intent real]"
{'response.redirected': False},"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 78.4.4143.75735 intent real]"


,browser
outcome_str,
{'response.redirected': False},"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox_beta Android 11 122.0b1 intent real]"
{'response.redirected': True},"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 78.4.4143.75735 intent real]"


,browser
outcome_str,
{'response.redirected': False},"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 78.4.4143.75735 intent real]"
{'response.redirected': True},"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox_beta Android 11 122.0b1 intent real]"


,browser
outcome_str,
{'response.redirected': False},"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox_beta Android 11 122.0b1 intent real]"
{'response.redirected': True},[opera Android 11 78.4.4143.75735 intent real]


,browser
outcome_str,
Fetch timed out,"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox_beta Android 11 122.0b1 intent real]"
{'response.redirected': True},"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 78.4.4143.75735 intent real]"


In [32]:
# Other approach to list the difference groups
def get_uniques(df):
    browsers = f"{df['browser'].unique().tolist()}"
    try:
        outcome = unique_outcomes[df.name[0]]
    except KeyError:
        outcome = {}
        #outcome = []
    outcome[browsers] = df.name[1]
    #outcome.append(f"{df.name[1]} ({browsers})")
    unique_outcomes[df.name[0]] = outcome

for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(grouping)
    unique_outcomes = {}
    group.groupby(["browser"])["outcome_str"].value_counts(normalize=False).reset_index().groupby(["outcome_str", "count"], group_keys=True).apply(get_uniques)
    d = pd.DataFrame.from_dict(unique_outcomes, orient="index")
    if d.shape[1] != 1:
        display(d)
        print()

('accesswindow_direct', 'direct')


,['firefox_beta Android 11 122.0b1 intent real'],"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless']",['opera Android 11 78.4.4143.75735 intent real'],['brave Android 11 1.60.116 intent real'],['chrome Android 11 120.0.6099.44 intent real'],"['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']"
{'window.open.opener': 'null'},12,24,28,40,44,56
"{'window.open.opener': 'object ""[object Window]""'}",56,44,40,28,24,12



('fetch_GET', 'credentials')


,"['brave Android 11 1.60.116 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 120.0.6099.44 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 78.4.4143.75735 intent real']","['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox_beta Android 11 122.0b1 intent real']",['opera Android 11 78.4.4143.75735 intent real'],"['brave Android 11 1.60.116 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 120.0.6099.44 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']"
"{'error': 'null', 'headers': 'content-length,'}",3.0,10,NaN,NaN
"{'error': 'null', 'headers': 'content-length,test,'}",19.0,20,NaN,NaN
"{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}",NaN,40,4.0,NaN
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",NaN,20,64.0,68.0



('fetch_GET', 'custom_headers')


...

,"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox_beta Android 11 122.0b1 intent real']","['brave Android 11 1.60.116 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 120.0.6099.44 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 78.4.4143.75735 intent real']"
error,32.0,NaN
load,NaN,32.0



('upgradeHSTS_direct', 'direct')


,['brave Android 11 1.60.116 intent real'],['opera Android 11 78.4.4143.75735 intent real'],"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox_beta Android 11 122.0b1 intent real']","['brave Android 11 1.60.116 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 120.0.6099.44 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 78.4.4143.75735 intent real']","['brave Android 11 1.60.116 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']",['chrome Android 11 120.0.6099.44 intent real'],"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless']",['firefox_beta Android 11 122.0b1 intent real'],"['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 78.4.4143.75735 intent real']"
Fetch timed out,1.0,2.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN
{'response.redirected': True},212.0,NaN,NaN,NaN,NaN,211.0,7.0,5.0,213.0
{'response.redirected': False},NaN,18.0,NaN,NaN,20.0,22.0,232.0,234.0,NaN
TypeError: Failed to fetch,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN



('upgradeHSTS_subdomain', 'subdomain')


,"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox_beta Android 11 122.0b1 intent real']","['brave Android 11 1.60.116 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 120.0.6099.44 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 78.4.4143.75735 intent real']",['opera Android 11 78.4.4143.75735 intent real'],"['brave Android 11 1.60.116 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 120.0.6099.44 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']"
Fetch timed out,12.0,NaN,NaN,NaN
{'response.redirected': False},54.0,NaN,14.0,15.0
{'response.redirected': True},3.0,NaN,37.0,36.0
TypeError: Failed to fetch,NaN,18.0,NaN,NaN


In [33]:
# Simmilarity between browsers and versions!
# (Current issue: displays a difference if one browser has no result at all; in the end we have 5+ stable results for each tests, thus it should not be an issue anymore)

# Function to calculate absolute and percentage overlap
def calculate_overlap(df, col1, col2):
    df = df[[col1, col2]].dropna()
    df.columns.values[0] = 0
    df.columns.values[1] = 1
    absolute_overlap = (df[0] == df[1]).sum()
    unique_rows = len(df)
    diff_rows = unique_rows - absolute_overlap
    percentage_diff = (diff_rows / unique_rows) * 100
    return diff_rows, percentage_diff, unique_rows


def display_overlap(df, show_all=True, name="All"):
    print(name)
    sim_frame = df.set_index(["test_id", "browser"])["outcome_str"].unstack().astype("str")
    sim_frame = sim_frame.reset_index().drop(columns=["test_id"])
    # Create a dictionary to store results
    results = {}
    matrix = {}
    
    # Iterate through all pairs of columns
    for i, col1 in enumerate(sim_frame.columns):
        for col2 in sim_frame.columns:
            key = f"{col1}_vs_{col2}"
            results[key] = calculate_overlap(sim_frame, col1, col2)
            try:
                l = matrix[col1]
            except KeyError:
                l = {}
            # 0 for absolute count of differences, 1 for percentage
            l[col2] = results[key][0]
            matrix[col1] = l
            
    
    # Convert the results to a DataFrame for better visualization
    results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Absolute Diff', 'Percentage Diff', "Count"])
    
    # Display difference matrix
    with pd.option_context("display.max_columns", 28):
        if show_all:
            # TODO: use a custom Order (e.g., all Android, or some chromium versions next to each other)
            display(pd.DataFrame(matrix))
        else:
            display(pd.DataFrame(matrix).drop_duplicates().T)


display_overlap(df)

for name, group in df.groupby("resp_type"):
    display_overlap(group, show_all=True, name=name)

for name, group in df.groupby("test_name"):
    display_overlap(group, show_all=False, name=name)

All


,brave Android 11 1.60.116 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 120.0.6099.44 intent real,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,firefox_beta Android 11 122.0b1 intent real,opera Android 11 78.4.4143.75735 intent real
brave Android 11 1.60.116 intent real,0,145,77,96,96,96,96,6955,6960,6961,3679
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,145,0,161,142,142,142,142,6891,6896,6923,3780
chrome Android 11 120.0.6099.44 intent real,77,161,0,21,21,21,21,6926,6931,6930,3660
chrome Ubuntu 22.04 120 selenium headless-new,96,142,21,0,2,2,2,6919,6924,6951,3657
chrome Ubuntu 22.04 121 selenium headless-new,96,142,21,2,0,0,0,6919,6924,6951,3657
chrome Ubuntu 22.04 122 selenium headless-new,96,142,21,2,0,0,0,6919,6924,6951,3657
edge Ubuntu 22.04 121 selenium headless-new,96,142,21,2,0,0,0,6919,6924,6951,3657
firefox Ubuntu 22.04 121 selenium headless,6955,6891,6926,6919,6919,6919,6919,0,21,82,6852
firefox Ubuntu 22.04 122 selenium headless,6960,6896,6931,6924,6924,6924,6924,21,0,75,6857
firefox_beta Android 11 122.0b1 intent real,6961,6923,6930,6951,6951,6951,6951,82,75,0,6879


basic


,brave Android 11 1.60.116 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 120.0.6099.44 intent real,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,firefox_beta Android 11 122.0b1 intent real,opera Android 11 78.4.4143.75735 intent real
brave Android 11 1.60.116 intent real,0,61,75,81,82,82,82,1038,1036,1051,395
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,61,0,77,71,72,72,72,1057,1055,1070,438
chrome Android 11 120.0.6099.44 intent real,75,77,0,6,7,7,7,1009,1007,1022,378
chrome Ubuntu 22.04 120 selenium headless-new,81,71,6,0,1,1,1,1015,1013,1028,384
chrome Ubuntu 22.04 121 selenium headless-new,82,72,7,1,0,0,0,1016,1014,1029,385
chrome Ubuntu 22.04 122 selenium headless-new,82,72,7,1,0,0,0,1016,1014,1029,385
edge Ubuntu 22.04 121 selenium headless-new,82,72,7,1,0,0,0,1016,1014,1029,385
firefox Ubuntu 22.04 121 selenium headless,1038,1057,1009,1015,1016,1016,1016,0,6,47,943
firefox Ubuntu 22.04 122 selenium headless,1036,1055,1007,1013,1014,1014,1014,6,0,49,941
firefox_beta Android 11 122.0b1 intent real,1051,1070,1022,1028,1029,1029,1029,47,49,0,955


parsing


...

,brave Android 11 1.60.116 intent real,firefox Ubuntu 22.04 121 selenium headless
brave Android 11 1.60.116 intent real,0,32
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,32
chrome Android 11 120.0.6099.44 intent real,0,32
chrome Ubuntu 22.04 120 selenium headless-new,0,32
chrome Ubuntu 22.04 121 selenium headless-new,0,32
chrome Ubuntu 22.04 122 selenium headless-new,0,32
edge Ubuntu 22.04 121 selenium headless-new,0,32
firefox Ubuntu 22.04 121 selenium headless,32,0
firefox Ubuntu 22.04 122 selenium headless,32,0
firefox_beta Android 11 122.0b1 intent real,32,0


upgradeHSTS_direct


,brave Android 11 1.60.116 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 120.0.6099.44 intent real,firefox Ubuntu 22.04 121 selenium headless,firefox_beta Android 11 122.0b1 intent real,opera Android 11 78.4.4143.75735 intent real
brave Android 11 1.60.116 intent real,0,2,2,246,246,6
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,2,0,2,244,246,6
chrome Android 11 120.0.6099.44 intent real,2,2,0,246,244,4
chrome Ubuntu 22.04 120 selenium headless-new,2,0,2,244,246,6
chrome Ubuntu 22.04 121 selenium headless-new,2,0,2,244,246,6
chrome Ubuntu 22.04 122 selenium headless-new,2,0,2,244,246,6
edge Ubuntu 22.04 121 selenium headless-new,2,0,2,244,246,6
firefox Ubuntu 22.04 121 selenium headless,246,244,246,0,2,250
firefox Ubuntu 22.04 122 selenium headless,246,244,246,0,2,250
firefox_beta Android 11 122.0b1 intent real,246,246,244,2,0,248


upgradeHSTS_subdomain


,brave Android 11 1.60.116 intent real,firefox Ubuntu 22.04 121 selenium headless,opera Android 11 78.4.4143.75735 intent real
brave Android 11 1.60.116 intent real,0,68,1
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,68,1
chrome Android 11 120.0.6099.44 intent real,0,68,1
chrome Ubuntu 22.04 120 selenium headless-new,0,68,1
chrome Ubuntu 22.04 121 selenium headless-new,0,68,1
chrome Ubuntu 22.04 122 selenium headless-new,0,68,1
edge Ubuntu 22.04 121 selenium headless-new,0,68,1
firefox Ubuntu 22.04 121 selenium headless,68,0,69
firefox Ubuntu 22.04 122 selenium headless,68,0,69
firefox_beta Android 11 122.0b1 intent real,68,0,69


In [37]:
# Limit on responses that have more than one outcome!
# This will remove all the branches that are the same in all browsers (and make trees large and confusing)
condition = df.groupby(["test_id"], observed=True)["outcome_str"].transform("nunique") != 1
tree_df = df.loc[condition]
print("Remove tests that are the same in all browsers", len(tree_df))

Remove tests that are the same in all browsers 77803


In [38]:
# How many rows exist for tree creation
with pd.option_context("display.max_rows", 100):
    display(tree_df[["test_name", "label", "relation_info"]].value_counts())

test_name                      label       relation_info 
oac_window.open                OAC         window.open       39182
fullscreen_iframe              PP          child_allow        5148
                                           direct             4554
upgradeHSTS_direct             HSTS        direct             2761
imgloading_iframe              CSP-IMG     direct             2673
framing_iframe                 CSP-FA      direct             2409
script_execution_iframe        CSP-SCRIPT  direct             2376
oac_iframe                     OAC         direct             1584
                                           sandbox            1584
subresourceloadingCOEP_img     COEP        direct             1551
referrer_iframe                RP          window.open        1320
                                           iframe             1320
framing_iframe                 CSP-FA      nested             1177
                               XFO         direct              814
    

In [40]:
h2o.init(nthreads=50, max_mem_size="100G", log_level="WARN")
h2o.no_progress()

base_dir = f"trees/{datetime.today().strftime('%Y-%m-%dT%H:%M')}" 

for group_name, group in tree_df.groupby(["test_name", "relation_info"]):
    print(group_name)
    pred_cols = ["browser", "org_origin", "resp_origin", "status_code", "raw_header"]
    group = group[["outcome_str", *pred_cols]]
    tree = make_tree(group, pred_cols, group_name, base_dir)
    # print(tree)


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,20 days 1 hour 6 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,4 months and 5 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


('accesswindow_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,20 days 1 hour 6 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,4 months and 5 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('accesswindow_direct', 'direct'), datapoints: 748
('fetch_GET', 'credentials')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


...

Create tree: ('subresourceloadingCORP_object', 'direct'), datapoints: 352
('upgradeHSTS_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,20 days 1 hour 6 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,4 months and 5 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.8 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('upgradeHSTS_direct', 'direct'), datapoints: 2761
('upgradeHSTS_subdomain', 'subdomain')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,20 days 1 hour 6 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,4 months and 5 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.8 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('upgradeHSTS_subdomain', 'subdomain'), datapoints: 759


# Manual analysis

In [377]:
# Chromium-based treats host-source as case sensitive even though it should not? (URLs and CSP is case-insensitive!)
# Play around: https://observer.sectec.rocks/opg/iframe/?url=https://echo.sectec.rocks/echo/?content-security-policy=img-src%20HTTPS://echo.sectec.rocks&ecocnt_css=%3Cimg%20src=https://echo.sectec.rocks/%3E&content-type=text/html
# Issue is in the scheme match which should be case-insensitive but is in Chromium-based browsers https://w3c.github.io/webappsec-csp/#match-schemes
# host-part matching was fixed here https://issues.chromium.org/issues/41412675, seems like they forgot fixing scheme-sensitivity 
df.loc[df["raw_header"].str.contains("img-src HTTPS://")][["clean_url", "raw_header", "outcome_str", "browser"]].value_counts().to_frame()

count
clean_url                                          raw_header                                         outcome_str browser                                                  
http://sub.headers.websec.saarland/_hp/tests/su... [['content-security-policy', 'img-src HTTPS://S... error       brave Android 11 1.60.116 intent real                   1
                                                                                                                  brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
                                                                                                                  chrome Android 11 120.0.6099.44 intent real             1
                                                                                                                  chrome Ubuntu 22.04 120 selenium headless-new           1
                                                                                                                  chrome Ubuntu 22.04 121 selenium headless-new           1
                                                                                                                  chrome Ubuntu 22.04 122 selenium headless-new           1
                                                                                                                  chrome iPadOS 17.3.1 122.0.6261.89 intent real          1
                                                                                                                  edge Ubuntu 22.04 121 selenium headless-new             1
                                                                                                                  firefox Ubuntu 22.04 121 selenium headless              1
                                                                                                                  firefox Ubuntu 22.04 122 selenium headless              1
                                                                                                                  firefox_beta Android 11 122.0b1 intent real             1
                                                                                                                  opera Android 11 78.4.4143.75735 intent real            1
                                                                                                                  safari macOS 14.3.1 17.3.1 selenium real                1
https://sub.headers.websec.saarland/_hp/tests/s... [['content-security-policy', 'img-src HTTPS://S... error       brave Android 11 1.60.116 intent real                   1
                                                                                                                  brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
                                                                                                                  chrome Android 11 120.0.6099.44 intent real             1
                                                                                                                  chrome Ubuntu 22.04 120 selenium headless-new           1
                                                                                                                  chrome Ubuntu 22.04 121 selenium headless-new           1
                                                                                                                  chrome Ubuntu 22.04 122 selenium headless-new           1
                                                                                                                  edge Ubuntu 22.04 121 selenium headless-new             1
                                                                                                                  opera Android 11 78.4.4143.75735 intent real            1
                                                                                                      load        chrome iPadOS 17.3.1 122.0.6261.89 intent real          1
                                      

In [378]:
bf = df.set_index(["test_id", "browser"])["outcome_str"].unstack().astype("str")

def com_browsers(df, b1, b2):
    bf["cr"] = bf[b1] != bf[b2]
    with pd.option_context("display.max_colwidth", None):
        display(bf.loc[bf["cr"]][[b1, b2]])

In [379]:
# Chromium 122 vs 121 vs 120 (desktop linux)
com_browsers(bf, "chrome Ubuntu 22.04 120 selenium headless-new", "chrome Ubuntu 22.04 121 selenium headless-new")
com_browsers(bf, "chrome Ubuntu 22.04 120 selenium headless-new", "chrome Ubuntu 22.04 122 selenium headless-new")
df.loc[df["response_id"] == 40058]["raw_header"].iloc[0]

browser,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new
test_id,,
referrer_iframe_iframe_https_sub.headers.websec.saarland_https_headers.webappsec.eu_40058_parsing,document.referrer: https://headers.webappsec.eu/,document.referrer:
referrer_iframe_window.open_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_196_basic,document.referrer: full_url,document.referrer: http://sub.headers.websec.saarland/


browser,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new
test_id,,
referrer_iframe_iframe_https_sub.headers.websec.saarland_https_headers.webappsec.eu_40058_parsing,document.referrer: https://headers.webappsec.eu/,document.referrer:
referrer_iframe_window.open_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_196_basic,document.referrer: full_url,document.referrer: http://sub.headers.websec.saarland/


"[['referrer-policy', 'no-referrer']]"

In [380]:
# Firefox 120 vs 121 (desktop linux)
com_browsers(bf, "firefox Ubuntu 22.04 121 selenium headless", "firefox Ubuntu 22.04 122 selenium headless")
df.loc[df["response_id"] == 65]["raw_header"].iloc[0]

browser,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless
test_id,,
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_headers.websec.saarland_61_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_61_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_61_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_https_sub.headers.websec.saarland_https_headers.webappsec.eu_18975_parsing,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_https_sub.headers.websec.saarland_https_headers.webappsec.eu_18977_parsing,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_https_sub.headers.websec.saarland_https_headers.webappsec.eu_18978_parsing,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_https_sub.headers.websec.saarland_https_headers.webappsec.eu_20261_parsing,"{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_https_sub.headers.websec.saarland_https_headers.webappsec.eu_20262_parsing,"{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_https_sub.headers.websec.saarland_https_headers.webappsec.eu_20263_parsing,"{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOr

'[]'

In [381]:
# Brave 121 vs Chrome 121
with pd.option_context("display.max_rows", 72):
    com_browsers(bf, "brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new", "chrome Ubuntu 22.04 121 selenium headless-new")

browser,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new
test_id,,
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.webappsec.eu_192_basic,document.referrer: http://headers.webappsec.eu/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.webappsec.eu_198_basic,document.referrer: http://headers.webappsec.eu/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.webappsec.eu_200_basic,document.referrer: http://headers.webappsec.eu/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.websec.saarland_192_basic,document.referrer: http://headers.websec.saarland/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.websec.saarland_198_basic,document.referrer: http://headers.websec.saarland/,document.referrer: full_url
...,...,...
referrer_iframe_window.open_https_sub.headers.websec.saarland_https_headers.websec.saarland_198_basic,document.referrer: https://headers.websec.saarland/,document.referrer: full_url
referrer_iframe_window.open_https_sub.headers.websec.saarland_https_headers.websec.saarland_200_basic,document.referrer: https://headers.websec.saarland/,document.referrer: full_url
referrer_iframe_window.open_https_sub.headers.websec.saarland_https_sub.sub.headers.websec.saarland_192_basic,document.referrer: https://sub.sub.headers.websec.saarland/,document.referrer: full_url


In [384]:
with pd.option_context("display.max_rows", 72):
    com_browsers(bf, "chrome iPadOS 17.3.1 122.0.6261.89 intent real", "safari macOS 14.3.1 17.3.1 selenium real")

browser,chrome iPadOS 17.3.1 122.0.6261.89 intent real,safari macOS 14.3.1 17.3.1 selenium real
test_id,,
accesswindow_direct_direct_http_sub.headers.websec.saarland_https_headers.webappsec.eu_21613_parsing,nan,"{'window.open.opener': 'object ""[object Window]""'}"
accesswindow_direct_direct_http_sub.headers.websec.saarland_https_headers.webappsec.eu_21615_parsing,nan,"{'window.open.opener': 'object ""[object Window]""'}"
accesswindow_direct_direct_http_sub.headers.websec.saarland_https_headers.webappsec.eu_21617_parsing,nan,"{'window.open.opener': 'object ""[object Window]""'}"
accesswindow_direct_direct_http_sub.headers.websec.saarland_https_headers.webappsec.eu_21619_parsing,nan,"{'window.open.opener': 'object ""[object Window]""'}"
accesswindow_direct_direct_http_sub.headers.websec.saarland_https_headers.webappsec.eu_21621_parsing,nan,"{'window.open.opener': 'object ""[object Window]""'}"
...,...,...
subresourceloadingCOEP_img_sandbox_http_sub.headers.websec.saarland_http_headers.webappsec.eu_59_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_sandbox_http_sub.headers.websec.saarland_http_headers.websec.saarland_59_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_sandbox_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_59_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"


In [385]:
pivot_df = df.pivot_table(index='test_name', columns='browser', values='test_id', aggfunc='count', fill_value=0)
# Calculate the difference between the counts of the first and second columns
pivot_df['Diff'] = pivot_df.iloc[:, 0] - pivot_df.iloc[:, 1]
display(pivot_df)

browser,brave Android 11 1.60.116 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 120.0.6099.44 intent real,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,chrome iPadOS 17.3.1 122.0.6261.89 intent real,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,firefox_beta Android 11 122.0b1 intent real,opera Android 11 78.4.4143.75735 intent real,safari macOS 14.3.1 17.3.1 selenium real,Diff
test_name,,,,,,,,,,,,,,
accesswindow_direct,5488,5488,5488,5488,5488,5488,4635,5488,5488,5488,5488,5488,5488,0
fetch_GET,48642,48642,48642,48642,48642,48642,48642,48642,48642,48642,48642,48642,48618,0
fetch_TEST,16214,16214,16214,16214,16214,16214,16214,16214,16214,16214,16214,16214,16206,0
framing_embed,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,0
framing_iframe,36800,36800,36800,36800,36800,36800,36800,36800,36800,36800,36800,36800,36800,0
framing_object,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,0
fullscreen_iframe,17056,17056,17056,17056,17056,17056,17056,17056,17056,17056,17056,17056,17056,0
imgloading_iframe,5692,5692,5692,5692,5692,5692,5692,5692,5692,5692,5692,5692,5692,0
oac_iframe,448,448,448,448,448,448,448,448,448,448,448,448,448,0
